# Import packages and intilize functions

In [ ]:
from __future__ import division, unicode_literals, print_function  # for compatibility with Python 2 and 3

import matplotlib as mpl
import matplotlib.pyplot as plt

import os
from pathlib import Path
import numpy as np
import pandas as pd
from pandas import DataFrame, Series  # for convenience

import pims
import trackpy as tp


# change the following to %matplotlib notebook for interactive plotting
%matplotlib inline

# Optionally, tweak styles.
mpl.rc('figure',  figsize=(10, 5))
mpl.rc('image', cmap='gray')

print('ready to go')

@pims.pipeline
def gray(image):
    return image[:, :]  # Take just the green channel


# Parameters to define

In [ ]:
## 1) Parameters governing loading and tracking
############################################

# Do the tracking, linking and filtering. Set these to false if you already performed the tracking and just want to analzye/plot differently
perform_tracking = False
perform_linking_trajectories = False
perform_filtering = False

# Directories
directory=r'C:\Users\Franz\OneDrive\_PhD\My_Papers\Volvox_Uncertainty_Minimization\Data\Figure10_Electroshocked_Phototaxis\lightSwtiching60sec_random_current3V'
save_dir=r'C:\Users\Franz\OneDrive\_PhD\My_Papers\Volvox_Uncertainty_Minimization\Individual_Graphs\\'

compress_images=True # True or False if to compress images for better and faster trajectory linking
compression_ratio=0.25 # compression ratio (0.1 means images are resized to a tenth of both x and y dimensions)
compresion_directory_name_ending='_compressed_0p25'
# Note 1: If you have already created the compressed images (because you have run it before), no need to change anything
#         it won't do it again and just juse those images. If you want to recompress images, just delete the directory 
#         that containsthe previously compressed images
# Note 2: For all the pixel sizes, midpoint calculations and chamber dimesions, use the original uncompressed image.
#         the script will automatically recalculate those based on the ratio. You might have to adjust the filter 
#         proporties though (see 2) below), as those are not that linearly calculated


analyze_subset=True # if this is True, it will only analyze frames starting at frames_to_analyze_start 
# going to frames_to_analyze_end. Check in the image sequence if there was any moving of the chamber at beginning or end.

frames_to_analyze_start=1 # first frame to analyze
frames_to_analyze_end=2990 # last frame to analyze

minimal_mass_base=1000 # threshold to use for mass (total brightness of an particle= number of pixels * their intensities)
# so for example, a 3x3 pixel sized volvox with each of a max intensity of 255 (8bit) would have a mass of 9*255=2295

Volvox_pixel_size_estimate=11 # err on the larger side

link_distance_pixels=15 # number of pixels to try link trajectories between frames (=distance Volvox move between frames)
# need only be larger than 10 if we are undersampling. the larger this distance, the longer the analysis takes. 

link_memory=5 # number of frames to try link trajectories (=number of frames Volvox could disappear in frames)
# need only be larger than 5 for partial illumination conditions like the fixed and random series, but 5 should work too 
# again,  the higher this value, the longer the analysis takes, and very large values will actually cause an error 

## 2) Parameters governing filtering trajectories by various characteristics
#########################################################################

filter_stubs_size=5 # filter out trajctories of things that only appear of x number of frames

# for mass and size, look at plot and imagej
minimal_mass=1000 # threshold to use for mass (total brightness of an particle= number of pixels * their intensities)

filter_by_size=False # True or False to filter by size
min_size=2
max_size=5

filter_by_ecc=False # True or False to filter by eccentricity (non circular shape)
max_ecc=0.3

filter_by_stds=True # True or False to filter by how much variation you would expect in position in x
# This filters out non-moving objects.
filter_std=5

filter_by_frame_count=False # True or False to filter by how many frames are contained in one trajectory
# This filters out Volvox that disappeared, but also removes broken up trajectories (i.e. if the linking failed)
filter_frame_count=50

## 3) Parameters for counting and plotting
#########################################################################

midpoint_x=2070  # midpoint in pixel x coordinates of the chamber width in the original, uncompressed image
midpoint_y=1120 # midpoint in pixel y coordinates of the chamber height in the original, uncompressed image

quartiles=True # if True, is counts volvox in quartile ends of the chamber instead of halfs for more prounced bias counts

chamber_width=3000 # chamber width in pixels of the chamber in the original, uncompressed image
chamber_height=2000 # chamber height in pixels of the chamber in the original, uncompressed image

save_plots=True # Do you want to save the plots? 

gaussian_smooth=True # True or False for gaussian smoothing of the Volvox count graph over time
sigma=15 # frames to average over for gaussian smoothing

fps=5 # frames per second of video taking (as in what was your framerate during acquisition)

plot_left_right_bias=False # True or False to plot the left-right bias counts
plot_top_bottom_bias=True  # True or False to plot the top-bottom bias counts

# Labels for the individual stimulation condtions (light pattern, anode/cathode side etc.)
label_left="Right half"
label_right="Left half"
label_top="Top Light"
label_bottom="Bottom Light"

# Compress Images

In [ ]:
from PIL import Image, ImageDraw

base_dir=str(Path(directory).parents[0]) 
sub_folder_name=os.path.basename(directory)  

if compress_images and perform_tracking:
    directory_new=directory+compresion_directory_name_ending
    if os.path.exists(directory_new)==False:
        os.mkdir(directory_new)
        for file_name in os.listdir(directory):
            #print("Processing %s" % file_name)
            img = Image.open(os.path.join(directory, file_name))
            x,y = img.size
            output = img.resize((int(x * compression_ratio), int(y* compression_ratio)), Image.ANTIALIAS)

            output_file_name = os.path.join(directory_new, file_name)
            output.save(output_file_name, "tiff", quality = 95)


# Get trajectories

In [ ]:
if not compress_images:
    directory_new=directory
else:
    directory_new=directory+compresion_directory_name_ending

base_dir=str(Path(directory_new).parents[0]) 
sub_folder_name=os.path.basename(directory_new)
    
if perform_tracking:
    print(directory_new)
    frames = gray(pims.open(directory_new+'/*.tiff'))
    plt.imshow(frames[frames_to_analyze_start])
    f = tp.locate(frames[frames_to_analyze_start],  int(Volvox_pixel_size_estimate), invert=False)
    f.head() 
    tp.annotate(f, frames[frames_to_analyze_start]);
    if analyze_subset:
        f = tp.batch(frames[frames_to_analyze_start:frames_to_analyze_end],  int(Volvox_pixel_size_estimate), minmass=minimal_mass_base, invert=False);
    else:
        f = tp.batch(frames, int(Volvox_pixel_size_estimate), minmass=minimal_mass, invert=False);

    base_dir=str(Path(directory_new).parents[0]) 
    sub_folder_name=os.path.basename(directory_new)

    f.to_csv(base_dir+'\\'+sub_folder_name+'_frames.csv')
    print(f.columns)
elif perform_linking_trajectories:
    f=pd.read_csv(base_dir+'\\'+sub_folder_name+'_frames.csv')

# Link trajectories

In [ ]:
if perform_linking_trajectories:
    t = tp.link_df(f, link_distance_pixels , memory=link_memory)

    t.to_csv(base_dir+'\\'+sub_folder_name+'_trajectories_raw.csv')
elif perform_filtering:
    t=pd.read_csv(base_dir+'\\'+sub_folder_name+'_trajectories_raw.csv')

# Filter particle trajectory by size, mass, eccentricity, stds, and frame count

In [ ]:
import math

if perform_filtering:
    t.head()
    t1 = tp.filter_stubs(t, filter_stubs_size)
    plt.figure()

    tp.mass_size(t1.groupby('particle').mean()); # convenience function -- just plots size vs. mass

    if filter_by_size:
        if filter_by_ecc:
            t2 = t1[((t1['mass'] > minimal_mass) & (t1['size'] > min_size)& (t1['size'] < max_size) & (t1['ecc'] < max_ecc))]
        else:    
            t2 = t1[((t1['mass'] > minimal_mass) & (t1['size'] > min_size)& (t1['size'] < max_size))]
    else:
        if filter_by_ecc:
            t2 = t1[((t1['mass'] > minimal_mass) & (t1['ecc'] < max_ecc))]
        else:    
            t2 = t1[((t1['mass'] > minimal_mass))]
    print('particles filter by size/mass/ecc =', len((list(set(t1.particle))))-len((list(set(t2.particle)))), 'out of', len((list(set(t1.particle)))))

    if filter_by_stds:

        t2p5=t2.copy()
        std=t2.groupby(['particle']).std()
        std.reset_index(level=0, inplace=True)

        list_set = set(t2.particle)
        unique_list = (list(list_set))
        unique_list.sort()
        for i in unique_list:
            std_x=float(std.loc[std.particle==i,'x'])
            if std_x<filter_std:
                t2p5=t2p5[t2p5['particle'] != i]
            elif math.isnan(std_x):
                t2p5=t2p5[t2p5['particle'] != i]

        len((list(set(t2p5.particle))))
        print('particles filter by stds =', len((list(set(t2.particle))))-len((list(set(t2p5.particle)))), 'out of', len((list(set(t2.particle)))))
        t2=t2p5.copy()



    t3=t2.copy()

    if filter_by_frame_count:
        list_set = set(t3.frame)
        unique_list = (list(list_set))
        unique_list.sort()
        for i in unique_list:
            t3i=t3[t3.frame==i]
            if t3i.frame.count()<filter_frame_count:
                t3=t3[t3['frame'] != i]

        print('particles filter by frame count =', len((list(set(t2.particle))))-len((list(set(t3.particle)))), 'out of', len((list(set(t2.particle)))))


    plt.figure()
    tp.annotate(t3[t3['frame'] == frames_to_analyze_start], frames[frames_to_analyze_start]);
    plt.figure()
    tp.plot_traj(t3);

    t4=t3.copy()

    t4.to_csv(base_dir+'\\'+sub_folder_name+'_trajectories_filtered.csv')
    
else:
    t4=pd.read_csv(base_dir+'\\'+sub_folder_name+'_trajectories_filtered.csv')

list_set = set(t4.frame)
unique_list = (list(list_set))
unique_list.sort()

# Count particles in each frame in each half of the chamber

In [ ]:
l=np.zeros(len(unique_list))
r=np.zeros(len(unique_list))

t=np.zeros(len(unique_list))
b=np.zeros(len(unique_list))

if not compress_images:
    compression_ratio=1
if quartiles:
    x1=int(midpoint_x*compression_ratio)-int(chamber_width*compression_ratio/4)
    x2=int(midpoint_x*compression_ratio)+int(chamber_width*compression_ratio/4)
    y1=int(midpoint_y*compression_ratio)-int(chamber_height*compression_ratio/4)
    y2=int(midpoint_y*compression_ratio)+int(chamber_height*compression_ratio/4)  
else:
    x1=int(midpoint_x*compression_ratio)
    x2=int(midpoint_x*compression_ratio)-1
    y1=int(midpoint_y*compression_ratio)
    y2=int(midpoint_y*compression_ratio)-1


# print(unique_list)
ii=0
for i in unique_list:
    t5=t4.loc[t4.frame==i]
    
    t_left=t5.loc[t5.x<x1]
    t_right=t5.loc[t5.x>x2]    
    
    t_top=t5.loc[t5.y<y1]
    t_bottom=t5.loc[t5.y>y2]

    l[ii]=t_left.shape[0]
    r[ii]=t_right.shape[0]
    
    
    t[ii]=t_top.shape[0]
    b[ii]=t_bottom.shape[0]

    ii=ii+1
    

print('mean # left : ',l.mean())
print('mean # right : ',r.mean())

print('mean # top : ',t.mean())
print('mean # bottom : ',b.mean())

# Smooth and plot trajectories

In [ ]:
import scipy as sp

if gaussian_smooth:
    l=sp.ndimage.gaussian_filter1d(l,sigma)
    r=sp.ndimage.gaussian_filter1d(r,sigma)
    t=sp.ndimage.gaussian_filter1d(t,sigma)
    b=sp.ndimage.gaussian_filter1d(b,sigma)

unique_list_time=np.array(unique_list)/fps

fig=plt.figure(figsize=(7,4))

if plot_left_right_bias:
    ax=plt.plot(unique_list_time, l, "-b", label=label_left)
    ax=plt.plot(unique_list_time, r, "--b", label=label_right)
    
if plot_top_bottom_bias:
    ax=plt.plot(unique_list_time, t, "-r", label=label_top)
    ax=plt.plot(unique_list_time, b, "--r", label=label_bottom)
ax=plt.legend(bbox_to_anchor=(1,1), loc="upper left")
ax=plt.xlabel('time (in s)')
ax=plt.ylabel('number of Volvox')
plt.show()

save_path=base_dir+'\\'+sub_folder_name+'_Volvox_counts_line_graph.png'

if save_plots:
    fig.savefig(save_path, bbox_inches='tight')

# Half only - Counting and Ploting Combined

In [ ]:
l=np.zeros(len(unique_list))
r=np.zeros(len(unique_list))

t=np.zeros(len(unique_list))
b=np.zeros(len(unique_list))

if not compress_images:
    compression_ratio=1

x1=int(midpoint_x*compression_ratio)
x2=int(midpoint_x*compression_ratio)-1
y1=int(midpoint_y*compression_ratio)
y2=int(midpoint_y*compression_ratio)-1


# print(unique_list)
ii=0
for i in unique_list:
    t5=t4.loc[t4.frame==i]
    
    t_left=t5.loc[t5.x<x1]
    t_right=t5.loc[t5.x>x2]    
    
    t_top=t5.loc[t5.y<y1]
    t_bottom=t5.loc[t5.y>y2]

    l[ii]=t_left.shape[0]
    r[ii]=t_right.shape[0]
    
    
    t[ii]=t_top.shape[0]
    b[ii]=t_bottom.shape[0]

    ii=ii+1
    

print('mean # left : ',l.mean())
print('mean # right : ',r.mean())

print('mean # top : ',t.mean())
print('mean # bottom : ',b.mean())

import scipy as sp

if gaussian_smooth:
    l=sp.ndimage.gaussian_filter1d(l,sigma)
    r=sp.ndimage.gaussian_filter1d(r,sigma)
    t=sp.ndimage.gaussian_filter1d(t,sigma)
    b=sp.ndimage.gaussian_filter1d(b,sigma)

unique_list_time=np.array(unique_list)/fps

fig=plt.figure(figsize=(7,4))

if plot_left_right_bias:
    ax=plt.plot(unique_list_time, l, "-b", label=label_left)
    ax=plt.plot(unique_list_time, r, "--b", label=label_right)
    
if plot_top_bottom_bias:
    ax=plt.plot(unique_list_time, t, "-r", label=label_top)
    ax=plt.plot(unique_list_time, b, "--r", label=label_bottom)
ax=plt.legend(bbox_to_anchor=(1,1), loc="upper left")
ax=plt.xlabel('time (in s)')
ax=plt.ylabel('number of Volvox')
plt.show()

save_path=base_dir+'\\'+sub_folder_name+'_Volvox_counts_line_graph_halfs.png'

if save_plots:
    fig.savefig(save_path, bbox_inches='tight')

# Quartile only - Counting and Ploting Combined

In [ ]:
l=np.zeros(len(unique_list))
r=np.zeros(len(unique_list))

t=np.zeros(len(unique_list))
b=np.zeros(len(unique_list))

if not compress_images:
    compression_ratio=1

x1=int(midpoint_x*compression_ratio)-int(chamber_width*compression_ratio/4)
x2=int(midpoint_x*compression_ratio)+int(chamber_width*compression_ratio/4)
y1=int(midpoint_y*compression_ratio)-int(chamber_height*compression_ratio/4)
y2=int(midpoint_y*compression_ratio)+int(chamber_height*compression_ratio/4)  


# print(unique_list)
ii=0
for i in unique_list:
    t5=t4.loc[t4.frame==i]
    
    t_left=t5.loc[t5.x<x1]
    t_right=t5.loc[t5.x>x2]    
    
    t_top=t5.loc[t5.y<y1]
    t_bottom=t5.loc[t5.y>y2]

    l[ii]=t_left.shape[0]
    r[ii]=t_right.shape[0]
    
    
    t[ii]=t_top.shape[0]
    b[ii]=t_bottom.shape[0]

    ii=ii+1
    

print('mean # left : ',l.mean())
print('mean # right : ',r.mean())

print('mean # top : ',t.mean())
print('mean # bottom : ',b.mean())

import scipy as sp

if gaussian_smooth:
    l=sp.ndimage.gaussian_filter1d(l,sigma)
    r=sp.ndimage.gaussian_filter1d(r,sigma)
    t=sp.ndimage.gaussian_filter1d(t,sigma)
    b=sp.ndimage.gaussian_filter1d(b,sigma)

unique_list_time=np.array(unique_list)/fps

fig=plt.figure(figsize=(7,4))

if plot_left_right_bias:
    ax=plt.plot(unique_list_time, l, "-b", label=label_left)
    ax=plt.plot(unique_list_time, r, "--b", label=label_right)
    
if plot_top_bottom_bias:
    ax=plt.plot(unique_list_time, t, "-b", label=label_top)
    ax=plt.plot(unique_list_time, b, "-r", label=label_bottom)
ax=plt.legend(bbox_to_anchor=(1,1), loc="upper left")
ax=plt.xlabel('time (in s)')
ax=plt.ylabel('number of Volvox')
plt.show()

save_path=base_dir+'\\'+sub_folder_name+'_Volvox_counts_line_graph_quartiles2.png'
if save_plots:
    fig.savefig(save_path, bbox_inches='tight')